# Image Preperation

To prepare our dataset for use we will need to do some cleaning and normalizing of the pictures. 
First, we will use the bounding box information we gathered earlier so that our photos will only
have dogs in them. Then we will resize the images so that they are uniform in addition to 
normalizing the colors. 

After we have created our high quality dog images we will do some image augmentation to help 
under represented breeds (n < 200) and breed categories (wild dogs and Foundation Stock Service).
We will do this using 


In [1]:
import pandas as pd
import os 
import xml.etree.ElementTree as ET
from PIL import Image, ImageOps
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import tensorflow.keras.layers as layers
import matplotlib.pyplot as plt

df = pd.read_csv('resources/csv/dog_annotations_with_groups.csv')
df.head()

,filename,breed_name,folder,xmin,ymin,xmax,ymax,pose,truncated,difficult,group_x,group_y,group
0,n02085620_10074.jpg,chihuahua,n02085620-Chihuahua,25,10,276,498,Unspecified,0,0,Toy,Toy,Toy
1,n02085620_10131.jpg,chihuahua,n02085620-Chihuahua,49,9,393,493,Unspecified,0,0,Toy,Toy,Toy
2,n02085620_10621.jpg,chihuahua,n02085620-Chihuahua,142,43,335,250,Unspecified,0,0,Toy,Toy,Toy
3,n02085620_1073.jpg,chihuahua,n02085620-Chihuahua,0,27,312,498,Unspecified,0,0,Toy,Toy,Toy
4,n02085620_10976.jpg,chihuahua,n02085620-Chihuahua,90,104,242,452,Unspecified,0,0,Toy,Toy,Toy


In [2]:
import os
from PIL import Image

error_log = []
cropped_folder = 'resources/stanford-dogs-dataset/cropped-images'
image_folder = 'resources/stanford-dogs-dataset/images'
breed_issues = {}

for _, row in df.iterrows():
    breed_name = row["breed_name"]
    folder_name = row["folder"].strip()  # Strip newlines & spaces
    filename = row["filename"].strip()  # Ensure filename is not empty

    # Check if filename is empty or incorrect
    if not filename or filename == "%s.jpg":
        # print(f"⚠️ Invalid filename for {breed_name} in {folder_name}")
        error_log.append(f"Invalid filename for {breed_name} in {folder_name}")
        continue  # Skip this entry

    img_path = os.path.join(image_folder, folder_name, filename)

    if not os.path.exists(img_path):
        # print(f"⚠️ Missing file: {img_path}")
        error_log.append(img_path)
        continue  # Skip missing images

    try: # Run of the mill image cropping routine
        # Open image
        img = Image.open(img_path)

        # Crop
        xmin, ymin, xmax, ymax = row["xmin"], row["ymin"], row["xmax"], row["ymax"]
        cropped_img = img.crop((xmin, ymin, xmax, ymax))

        # Ensure breed folder exists
        breed_cropped_path = os.path.join(cropped_folder, breed_name)
        os.makedirs(breed_cropped_path, exist_ok=True)

        # Save cropped image
        save_path = os.path.join(breed_cropped_path, filename)
        cropped_img.save(save_path)

    except Exception as e:
        error_log.append(f"{img_path} - {e}")

# Save error log
with open("missing_images_log.txt", "w") as f:
    for entry in error_log:
        f.write(f"{entry}\n")

print(f"✅ Cropping complete! {len(error_log)} issues logged.")

✅ Cropping complete! 1 issues logged.


In [3]:
expected_breeds = set(df["breed_name"].unique())
actual_breeds = set(os.listdir(cropped_folder))

# Check which breeds are missing from the folders
missing_folders = expected_breeds - actual_breeds
extra_folders = actual_breeds - expected_breeds

print("📌 Breeds in dataset but missing from image folders:", missing_folders)
print("📌 Extra breed folders that don't match dataset:", extra_folders)

📌 Breeds in dataset but missing from image folders: set()
📌 Extra breed folders that don't match dataset: set()


In [13]:
total_imgs = 0
for breed in os.listdir(cropped_folder):
    breed_path = os.path.join(cropped_folder, breed)
    num_images = len(os.listdir(breed_path))
    total_imgs += num_images

print(f"There are {total_imgs} cropped images")
print(f"There are {len(df['breed_name']) - total_imgs} missing images.")


There are 20579 cropped images
There are 1547 missing images.


Ok so we are missing 10% of our data for some reason. I am dealing with that later. 
## Image Normalization

In [14]:
# Define normalization and augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=15,  # Rotate up to ±15 degrees
    width_shift_range=0.1,  # Shift image width by 10%
    height_shift_range=0.1,  # Shift image height by 10%
    shear_range=0.1,  # Shear transform
    zoom_range=0.2,  # Zoom in/out by up to 20%
    horizontal_flip=True,  # Flip images horizontally
    brightness_range=[0.8, 1.2],  # Random brightness adjustment
    fill_mode='nearest'  # Fill in missing pixels after transformation
)

# Apply transformations to dataset
train_generator = datagen.flow_from_directory(cropped_folder, target_size=(128,128), batch_size=32)


Found 20579 images belonging to 120 classes.


In [15]:
def resize_with_padding(image, target_size=(128, 128)):
    """Resize an image while keeping aspect ratio and adding padding."""
    image = np.array(image)  # Ensure it's a NumPy array
    image = (image * 255).astype(np.uint8)  # Convert to uint8

    img = Image.fromarray(image)  # Convert to PIL Image
    img = ImageOps.fit(img, target_size, method=Image.Resampling.LANCZOS, centering=(0.5, 0.5))

    return np.array(img)  # Convert back to NumPy array


# Define a custom preprocessing function for ImageDataGenerator
def preprocess_image(image):
    """Resize with padding and normalize pixel values."""
    image = resize_with_padding(image)  # Resize and pad
    image = image / 255.0  # Normalize pixels to [0,1]
    return image

# Apply it in ImageDataGenerator
datagen = ImageDataGenerator(preprocessing_function=preprocess_image)

train_generator = datagen.flow_from_directory(
    cropped_folder,  # Your dataset path
    target_size=(128, 128),  # The final target size
    batch_size=64
)


Found 20579 images belonging to 120 classes.


In [16]:
batch_images, batch_labels = next(train_generator)

print(f"Image batch shape: {batch_images.shape}")  # Should be (batch_size, 128, 128, 3)
print(f"Label batch shape: {batch_labels.shape}")  # Should match the number of classes
print(f"Pixel range: min={batch_images.min()}, max={batch_images.max()}")  # Should be between 0 and 1


Image batch shape: (64, 128, 128, 3)
Label batch shape: (64, 120)
Pixel range: min=0.0, max=1.0


In [17]:
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_image,
    validation_split=0.2  # 20% validation
)

train_generator = datagen.flow_from_directory(
    cropped_folder,
    target_size=(128, 128),
    batch_size=64,
    subset='training'
)

val_generator = datagen.flow_from_directory(
    cropped_folder,
    target_size=(128, 128),
    batch_size=64,
    subset='validation'
)


Found 16507 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


In [9]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(116, activation='softmax')  # 116 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()  # Print model structure


NameError: name 'models' is not defined

In [47]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10  # Adjust as needed
)


Epoch 1/10
251/251 [==============================] - 149s 591ms/step - loss: 4.5956 - accuracy: 0.0290 - val_loss: 4.3743 - val_accuracy: 0.0486
Epoch 2/10
251/251 [==============================] - 154s 613ms/step - loss: 3.9756 - accuracy: 0.0981 - val_loss: 4.0302 - val_accuracy: 0.0896
Epoch 3/10
251/251 [==============================] - 156s 621ms/step - loss: 3.2242 - accuracy: 0.2316 - val_loss: 3.9100 - val_accuracy: 0.1167
Epoch 4/10
251/251 [==============================] - 174s 692ms/step - loss: 2.0353 - accuracy: 0.4866 - val_loss: 4.6806 - val_accuracy: 0.1172
Epoch 5/10
251/251 [==============================] - 177s 706ms/step - loss: 0.8221 - accuracy: 0.7858 - val_loss: 6.6313 - val_accuracy: 0.1142
Epoch 6/10
251/251 [==============================] - 241s 959ms/step - loss: 0.2708 - accuracy: 0.9300 - val_loss: 8.4326 - val_accuracy: 0.1003
Epoch 7/10
251/251 [==============================] - 179s 709ms/step - loss: 0.1442 - accuracy: 0.9667 - val_loss: 9.2541 -

In [48]:
val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc*100:.2f}%")

model.save("dog_breed_classifier.h5")

62/62 [==============================] - 13s 210ms/step - loss: 10.4139 - accuracy: 0.1094
Validation Accuracy: 10.94%
